# Ficheux 2020 single-qubit phases
In this tutorial we show how to ideally correct for single-qubit phases accumulated during the implementation of the CPHASE gate between two fluxonia using the Ficheux scheme. The notebook is adapted from the notebook Ficheux 2020.

In [1]:
import numpy as np
import sys
import time
import datetime
import qutip as qtp 
import cmath
import matplotlib.pyplot as plt
from scipy import constants
from scipy.optimize import minimize
import pysqkit
from pysqkit.util.linalg import get_mat_elem, tensor_prod
from pysqkit.solvers import solvkit
from typing import List
import matplotlib
matplotlib.rcParams['mathtext.fontset'] = 'cm'
from IPython.display import display, Latex
from itertools import product, combinations

The gate is based on the following driven Hamiltonian of two-capacitively coupled fluxonium qubits

$$\hat{H} = \hat{H}^{(0)}_A + \hat{H}^{(0)}_B + \hat{V} + \hat{H}_{drive}$$

$$\hat{H}^{(0)}_A = 4 E_{C,A} \cdot \hat{n}^2_A + \frac{1}{2}E_{L,A}\cdot\hat{\varphi}^2_A - E_{J,A}\cos\left(\hat{\varphi}_A - \phi_{ext,A}\right)$$

$$\hat{H}^{(0)}_B = 4 E_{C,B} \cdot \hat{n}^2_B + \frac{1}{2}E_{L,B}\cdot\hat{\varphi}^2_B - E_{J,B}\cos\left(\hat{\varphi}_B - \phi_{ext,B}\right)$$

$$\hat{V} = J_C \cdot \hat{n}_A \cdot \hat{n}_B$$

$$\frac{\hat{H}_{drive}}{h} = \left( \epsilon_A \hat{n}_A + \epsilon_B \hat{n}_B \right) \cos(2\pi f_d t)$$

Energy spectrum :
![](energy_levels.jpg)

# Single fluxonia
We take parameters as in Table 1 of the paper. 

In [2]:
# Values in GHz (divided by h)
#Fluxonium A
ec_a = .973
el_a = .457
ej_a = 5.899 
flux_a = 1/2#pi in the cos

# Fluxonium B 
ec_b = 1.027
el_b = 0.684
ej_b = 5.768
flux_b = 1/2

n_fock = 100

In [3]:
levels = 5
flx_a = pysqkit.qubits.Fluxonium('A', ec_a, el_a, ej_a, flux_a, dim_hilbert=n_fock)
flx_b = pysqkit.qubits.Fluxonium('B', ec_b, el_b, ej_b, flux_b, dim_hilbert=n_fock)
flx_a.diagonalize_basis(levels)
flx_b.diagonalize_basis(levels)

We obtain the energy levels and eigenstates of the two fluxonia

In [4]:
energies_a, eig_states_a = flx_a.eig_states(levels)
energies_b, eig_states_b = flx_b.eig_states(levels)

In [5]:
energies_in_lexico = np.zeros(levels**2, dtype=float)
count = 0
for k in range(0, levels):
    for m in range(0, levels):
        energies_in_lexico[count] = energies_b[m] + energies_a[k]
        count += 1
ascending_to_lexico = np.argsort(energies_in_lexico)

label_converter = ascending_to_lexico
label_converter

array([ 0,  5,  1,  6,  2,  7, 10, 11, 15, 16,  3,  8, 20, 21,  4,  9, 12,
       17, 13, 18, 22, 14, 23, 19, 24], dtype=int64)

In [6]:
energies_in_lexico = np.zeros(levels**2, dtype=float)
count = 0
for k in range(0, levels):
    for m in range(0, levels):
        energies_in_lexico[count] = energies_b[m] + energies_a[k]
        count += 1
ascending_to_lexico = np.argsort(energies_in_lexico)

label_converter = ascending_to_lexico
label_converter

array([ 0,  5,  1,  6,  2,  7, 10, 11, 15, 16,  3,  8, 20, 21,  4,  9, 12,
       17, 13, 18, 22, 14, 23, 19, 24], dtype=int64)

In [7]:
# This function allows to return the energy of the "adiabatically continued" coupled levels
def label_to_energies(
    k: int, 
    m: int,
    levels: int,
    energies: np.ndarray,  #in ascending order
    label_converter: np.ndarray 
) -> float:
    label = k*levels + m
    index = np.where(label_converter == label)[0][0] #index such that converter[index] = label
    return energies[index]

def label_to_states(
    k: int, 
    m: int,
    levels: int,
    eig_states: np.ndarray,
    label_converter: np.ndarray 
) -> float:
    label = k*levels + m
    index = np.where(label_converter == label)[0][0] #index such that converter[index] = label
    return eig_states[index] 

def energy_ascending_order_to_label(
    k : int,
    levels: int,
    label_converter : np.ndarray):
    
    lexico_rank = label_converter[k]
    i = lexico_rank//levels
    j = lexico_rank%levels
    
    return i, j

# Adding drives

In [8]:
flx_a.add_drive(pysqkit.drives.gaussian_microwave_drive)
flx_b.add_drive(pysqkit.drives.gaussian_microwave_drive)

AttributeError: module 'pysqkit.drives' has no attribute 'gaussian_microwave_drive'

# Coupled fluxonia

In [ ]:
jc = 0.224 #GHz (divided by h)

coupled_flx = flx_a.couple_to(flx_b, coupling=pysqkit.couplers.capacitive_coupling, strength=jc)
energies_sys, eigstates_sys = coupled_flx.eig_states() 

coupled_flx_bare = flx_a.couple_to(flx_b, coupling=pysqkit.couplers.capacitive_coupling, strength=0)
energies_bare, eigstates_bare = coupled_flx_bare.eig_states() 

## Drive

In [ ]:
eig_states_by_label = []
energies_by_label = []

for i_a in range(levels):
    energies_by_label.append([])
    eig_states_by_label.append([])
    for i_b in range(levels):
        eig_states_by_label[i_a].append(qtp.Qobj(inpt=  label_to_states(i_a, i_b, levels, eigstates_sys, label_converter), 
                   dims=[[levels, levels], [1, 1]], shape=[levels*levels, 1]))
        energies_by_label[i_a].append(label_to_states(i_a, i_b, levels, energies_sys, label_converter))

In [ ]:
n_a = qtp.tensor(flx_a.charge_op(as_qobj = True), qtp.qeye(flx_b.charge_op(as_qobj = True).dims[0]))
n_b = qtp.tensor(qtp.qeye(flx_a.charge_op(as_qobj = True).dims[0]), flx_b.charge_op(as_qobj = True))
n_charge = [n_a, n_b]

In [ ]:
big_delta = ( label_to_energies(2, 1, levels, energies_sys, label_converter)   - \
              label_to_energies(1, 1, levels, energies_sys, label_converter) ) - \
            ( label_to_energies(2, 0, levels, energies_sys, label_converter)   - \
              label_to_energies(1, 0, levels, energies_sys, label_converter) )

display(Latex(r'$\Delta/2\pi $ = {:.3f} MHz'.format(big_delta*1000)))

According to the paper we have :

$$\Omega_{11-21}  = \left| \epsilon_A \langle 11 | \hat{n}_A | 21 \rangle + \epsilon_B \langle 11 | \hat{n}_B | 21 \rangle \right|$$
$$\Omega_{10-20}  = \left| \epsilon_A \langle 10 | \hat{n}_A | 20 \rangle + \epsilon_B \langle 10 | \hat{n}_B | 20 \rangle \right|$$

And then :
$$r = \dfrac{\Omega_{11-21}}{\Omega_{10-20}} \text{  and  } \dfrac{\delta}{\Delta} = \dfrac{r^2 - \sqrt{(r^2-1)^2 + r^2 }} {r^2 - 1}$$ 

With these conventions taken in the paper :
$$\Delta = f_{11-21} - f_{10-20}$$
$$\delta = f_{11-21} - f_d$$

With this $\delta$, we have $\Omega = \Delta$ and a CZ gate is achieved with zero leakage in time exactly $t_{drive} = 1/\Delta$. 

We choose as in the paper to have : $\dfrac{\epsilon_A}{\epsilon_B} = .9$

In [ ]:
ratio_eps = .9

We now match the conditions

In [ ]:
def Omega1(eps_b, ratio_eps):
    eps_a = eps_b*ratio_eps
    eps = [eps_a, eps_b]
    
    rabi_10_20 = np.abs(eps[0] * (eig_states_by_label[1][0].dag() * n_charge[0] * eig_states_by_label[2][0]).full()[0, 0] + \
                              eps[1] * (eig_states_by_label[1][0].dag() * n_charge[1] * eig_states_by_label[2][0]).full()[0, 0] )
    rabi_11_21 = np.abs(eps[0] * (eig_states_by_label[1][1].dag() * n_charge[0] * eig_states_by_label[2][1]).full()[0, 0] + \
                              eps[1] * (eig_states_by_label[1][1].dag() * n_charge[1] * eig_states_by_label[2][1]).full()[0, 0] )

    rabi_ratio = rabi_11_21/rabi_10_20
    ratio_delta = (rabi_ratio**2 - np.sqrt((rabi_ratio**2 - 1)**2 + rabi_ratio**2)) / (rabi_ratio**2 - 1)
    
    big_delta = ( label_to_energies(2, 1, levels, energies_sys, label_converter)   - \
                  label_to_energies(1, 1, levels, energies_sys, label_converter) ) - \
                ( label_to_energies(2, 0, levels, energies_sys, label_converter)   - \
                  label_to_energies(1, 0, levels, energies_sys, label_converter) )
    small_delta = big_delta * ratio_delta
    
    return np.sqrt(rabi_11_21**2 + small_delta**2)

def Omega2(eps_b, ratio_eps):
    eps_a = eps_b*ratio_eps
    eps = [eps_a, eps_b]
        
    rabi_10_20 = np.abs(eps[0] * (eig_states_by_label[1][0].dag() * n_charge[0] * eig_states_by_label[2][0]).full()[0, 0] + \
                              eps[1] * (eig_states_by_label[1][0].dag() * n_charge[1] * eig_states_by_label[2][0]).full()[0, 0] )
    rabi_11_21 = np.abs(eps[0] * (eig_states_by_label[1][1].dag() * n_charge[0] * eig_states_by_label[2][1]).full()[0, 0] + \
                              eps[1] * (eig_states_by_label[1][1].dag() * n_charge[1] * eig_states_by_label[2][1]).full()[0, 0] )

    rabi_ratio = rabi_11_21/rabi_10_20
    ratio_delta = (rabi_ratio**2 - np.sqrt((rabi_ratio**2 - 1)**2 + rabi_ratio**2)) / (rabi_ratio**2 - 1)
    
    big_delta = ( label_to_energies(2, 1, levels, energies_sys, label_converter)   - \
                  label_to_energies(1, 1, levels, energies_sys, label_converter) ) - \
                ( label_to_energies(2, 0, levels, energies_sys, label_converter)   - \
                  label_to_energies(1, 0, levels, energies_sys, label_converter) )
    small_delta = big_delta * ratio_delta
    
    return np.sqrt(rabi_10_20**2 + (small_delta - big_delta)**2)

In [ ]:
rg_e_b = np.linspace(0.01, 1, 1000)

plt.figure(figsize=(8, 6))
plt.plot(rg_e_b, Omega1(rg_e_b, ratio_eps), color = 'blue', label = 'O1')
plt.plot(rg_e_b, Omega2(rg_e_b, ratio_eps), color = 'red', label = 'O2')
plt.plot(rg_e_b, [big_delta]*len(rg_e_b), color = 'orange', label = 'big Delta')

plt.legend()
plt.show()

## Rest of parameters

In [ ]:
eps_b = 0.03634
eps_a = ratio_eps*eps_b
eps = [eps_a, eps_b]

In [ ]:
#we calculate rabi freq amplitudes :

rabi_10_20 = np.abs(eps[0] * (eig_states_by_label[1][0].dag() * n_charge[0] * eig_states_by_label[2][0]).full()[0, 0] + \
                          eps[1] * (eig_states_by_label[1][0].dag() * n_charge[1] * eig_states_by_label[2][0]).full()[0, 0] )

rabi_11_21 = np.abs(eps[0]*(eig_states_by_label[1][1].dag() * n_charge[0] * eig_states_by_label[2][1]).full()[0, 0] + \
                          eps[1]*(eig_states_by_label[1][1].dag() * n_charge[1] * eig_states_by_label[2][1]).full()[0, 0] )

rabi_ratio = rabi_11_21/rabi_10_20
display(Latex(r'$r$ = {:.3f}'.format(rabi_ratio)))

ratio_delta = (rabi_ratio**2 - np.sqrt((rabi_ratio**2 - 1)**2 + rabi_ratio**2)) / (rabi_ratio**2 - 1)
display(Latex(r'$\delta/\Delta$ = {:.3f}'.format(ratio_delta)))

In [ ]:
small_delta = big_delta*ratio_delta

freq_drive = ( label_to_energies(2, 1, levels, energies_sys, label_converter)   - \
               label_to_energies(1, 1, levels, energies_sys, label_converter) )  - small_delta
t_drive = 1/big_delta

display(Latex(r'$\Delta/2\pi $ = {:.3f} MHz'.format(big_delta*1000)))
display(Latex(r'$\delta/2\pi $ = {:.3f} MHz'.format(small_delta*1000)))
display(Latex(r'$\omega_d/2\pi $ = {:.3f} GHz'.format(freq_drive)))

In [ ]:
rabi_freq = big_delta

rabi_period = 1/rabi_freq # [ns]
display(Latex(r'$T_{{Rabi}} $ = {:.3f} ns'.format(rabi_period)))

#  Simulations

In [ ]:
#AC: remark. I am starting to think that the pulse is nor correctly implemented
# In fact it is discontinuous at t = rise_time which is not possible. To make it continuous
# at the plateau one should set the amplitude not to be one, but (1 - exp(-rise_time)**2(2*sigma**2))
# In the paper this is a bit confusing I would say.

def pulse_function(
    t: float,
    rise_time: float,
    plateau_time: float,
    freq_drive: float
) -> float:
    sigma = rise_time/np.sqrt(2*np.pi)
    flat_amplitude = (1 - np.exp(-rise_time**2/(2*sigma**2)))
    if t < rise_time:
        return 1/flat_amplitude*(np.exp(-(t - rise_time)**2/(2*sigma**2)) - \
                                 np.exp(-rise_time**2/(2*sigma**2)))*np.cos(2*np.pi*freq_drive*t)
    elif t < rise_time + plateau_time:
        return np.cos(2*np.pi*freq_drive*t)
    else:
        return 1/flat_amplitude*(np.exp(-(t - plateau_time - rise_time)**2/(2*sigma**2)) - \
                          np.exp(-rise_time**2/(2*sigma**2)))*np.cos(2*np.pi*freq_drive*t)

def rabi_drive(
     t: float,
    freq_drive: float
) -> float:
    return np.cos(2*np.pi*freq_drive*t)
    

In [ ]:
t_rise = 17.6
t_tot = 58.3
t_plateau = t_tot - 2*t_rise
#simulation setup (noiseless)
pts_per_drive_period = 15
simu_time = t_tot

nb_points = int( simu_time * freq_drive * pts_per_drive_period  )
tlist = np.linspace(0, simu_time, nb_points)
pulse_drive = np.zeros(nb_points, dtype=float)

interest_levels = [[0,0], [0,1], [1,0], [1,1], [0,2], [1,2], [2,0], [2,1]]
labels = ["$| "+ str(k) + str(m) + " \\rangle $" for k,m in interest_levels]

hamil0 = coupled_flx.hamiltonian(as_qobj=True)
# hamil_drive = eps[0]*n_charge[0] + eps[1]*n_charge[1] 

# for i in range(0, nb_points):
#     pulse_drive[i] = pulse_function(tlist[i], t_rise, t_plateau, freq_drive)
#     #pulse_drive[i] = rabi_drive(tlist[i], freq_drive)
coupled_flx['A'].drives[0].set_params(phase=0, amp=eps_a, 
                                      plateau_time=t_plateau, rise_time=t_rise, freq=freq_drive)
coupled_flx['B'].drives[0].set_params(phase=0, amp=eps_b, 
                                      plateau_time=t_plateau, rise_time=t_rise, freq=freq_drive)
drive_hamils = [coupled_flx['A'].drives[0].hamiltonian(as_qobj=True)]
drive_hamils.append(coupled_flx['B'].drives[0].hamiltonian(as_qobj=True))
#drive_hamils = [drive.hamiltonian(as_qobj=True) for qubit in coupled_flx for drive in qubit.drives]
#pulses = [drive.eval_pulse(time=tlist) for qubit in coupled_flx for drive in qubit.drives]

In [ ]:
drive_pulse_a = np.zeros(nb_points)
drive_pulse_b = np.zeros(nb_points)
for i in range(0, nb_points):
    drive_pulse_a[i] = coupled_flx['A'].drives[0].eval_pulse(time=tlist[i])
    drive_pulse_b[i] = coupled_flx['B'].drives[0].eval_pulse(time=tlist[i])
pulses = [drive_pulse_a, drive_pulse_b]

In [ ]:
state_in = []
outputs = []
for i in range(0, 2):
    for k in range(0, 2):
        state_in.append(eig_states_by_label[i][k])
for i in range(0, 4):
    outputs.append(solvkit.integrate(tlist*2*np.pi, state_in[i], hamil0, drive_hamils, pulses, [], "mesolve"))

In [ ]:
statein_label = '11'
statein_index = int(statein_label, 2)
rhos_by_label = []
for i in range(len(eig_states_by_label)):
    rhos_by_label.append([])
    for j in range(len(eig_states_by_label[0])):
        rhos_by_label[i].append(eig_states_by_label[i][j]*eig_states_by_label[i][j].dag())  
        
probs = np.zeros([len(labels), nb_points], dtype=float)
for k in range(0, len(labels)):
    i, j = interest_levels[k]
    for m in range(0, nb_points):        
        probs[k, m] = qtp.expect(rhos_by_label[i][j], outputs[statein_index].states[m]) 

In [ ]:
plot_setup = {'lw': 2.0, 'fs': 16, 'fs_text': 14, 'ls': 14}
def plot_probas(
    tlist: np.ndarray, 
    probs: np.ndarray,
    statein_label: str,
    opt: dict
) -> None:
    fig, ax = plt.subplots(int(np.ceil(len(labels)/4)), 4, figsize = (16, 8))
    for k in range(0, len(labels)):
        ax[k//4, k%4].plot(tlist, probs[k, :], linewidth=opt['lw'])

        ax[k//4, k%4].set_ylim(0, 1.0)
        ax[k//4, k%4].set_title(labels[k], fontsize=opt['fs'])
        ax[k//4, k%4].set_ylim(-0.1, 1.1)
        ax[k//4, k%4].axhline(y=0, color='grey', linestyle='--')
        ax[k//4, k%4].axhline(y=1, color='grey', linestyle='--')

        if k%4 != 0:
            ax[k//4, k%4].axes.yaxis.set_ticklabels([])

    fig = plt.gcf()
    fig.suptitle('$\mathrm{Probabilities \ of \ being \ in \ each \ eigenstate \
                 \ as \ a \ function \ of \ time \ [ns] \ with \ initial \ state}$' + 
                 ' $\\vert ' + statein_label + ' \\rangle$', fontsize=opt['fs']) 
    plt.show()

plot_probas(tlist, probs, statein_label, plot_setup)

In [ ]:
u_gate = np.zeros([4, 4], dtype=complex)
for i in range(0, 4):
    for k in range(0, 4):
        u_gate[i, k] = (state_in[i].dag()*outputs[k].states[nb_points - 1])[0, 0]
u_ideal_phases = np.zeros([4, 4], dtype=complex)
for i in range(0, 2):
    for k in range(0, 2):
        n = i*2 + k
        u_ideal_phases[n, n] = np.exp(-1j*2*np.pi*energies_by_label[i][k]*simu_time)

#taking out phase associated with 00 from numerical simulations
u_ideal_phases = np.conj(u_gate[0, 0])*u_ideal_phases
u_gate = np.conj(u_gate[0, 0])*u_gate 

We now detail how to obtain the single-qubit gates in order to bring the (ideal) gate to the CZ form
$$
\mathrm{CZ} = \begin{pmatrix}
1 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & -1
\end{pmatrix}
$$
The Ficheux protocol we have just simulated realizes (ideally) a unitary 
$$
U =  \begin{pmatrix}
1 & 0 & 0 & 0 \\
0 & e^{i \phi_{01}} & 0 & 0 \\
0 & 0 & e^{i \phi_{10}} & 0 \\
0 & 0 & 0 & e^{i \phi_{11}}
\end{pmatrix}
$$
where the phases satisfy the condition
$$
\phi_{11} - \phi_{01} - \phi_{10} = \pi \mod 2 \pi.
$$
Let us define a generic single-qubit phase gate
$$
P(\phi) = \begin{pmatrix}
1 & 0 \\
0 & e^{i \phi}
\end{pmatrix}.
$$
The single-qubit gates to be applied are $P(-\phi_{01})$ on the second qubit and $P(-\phi_{10})$ on the first qubit (to see it more clearly apply them sequentially). In fact, one can check that 
$$
P(-\phi_{10}) \otimes P(-\phi_{01}) U = \begin{pmatrix}
1 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & e^{i (\phi_{11} - \phi_{10} - \phi_{01})}
\end{pmatrix},
$$
which given the previous condition on the phases is equal to the CZ gate.


I remark that this way of obtaining the phases assumes the ideality of the model. This does not mean that is the optimal way. For instance I noticed that (quite surprisingly) the ideal phase acquired by $\lvert 00 \rangle$ is not the same as the one acquired in the simulations, but there is a small deviation

In [ ]:
display(Latex(r'$\phi_{{00}}^{{\mathrm{{ideal}}}} $ = {:.3f} '.format(cmath.phase(u_ideal_phases[0, 0]))))
display(Latex(r'$\phi_{{00}} $ = {:.3f} '.format(cmath.phase(u_gate[0, 0]))))

I interpret this as an effect of the drive that is dressing the states. This effect might or might not be correctable by single-qubit gates, but they must be optimized and this is not included in the previous ideal analysis.

In [ ]:
def single_qubit_correction(
    u_gate: np.ndarray
) -> np.ndarray:
    phi_10 = cmath.phase(u_gate[2, 2])
    phi_01 = cmath.phase(u_gate[1, 1])
    p_phi10 = np.array([[1, 0], [0, np.exp(-1j*phi_10)]])
    p_phi01 = np.array([[1, 0], [0, np.exp(-1j*phi_01)]])
    return tensor_prod([p_phi10, p_phi01])

In [ ]:
cz = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, -1]])
u_correction = single_qubit_correction(u_gate)
u_ideal = qtp.Qobj(u_correction.conj().T.dot(cz))
sup_u_ideal = qtp.to_super(u_ideal)